<a href="https://colab.research.google.com/github/michaelrosen3/kirby_index/blob/main/calculating_release_angles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here are instructions for calculating release angles for all pitches during the 2024 season in 10 minutes using Python.

In [ ]:
# If you're in Google Colab for the first time, you need to remove the hashtag from in front of the next line of code and press enter:
# pip install pybaseball

In [ ]:
# These two functions calculate vertical and horizontal release angles based on the 9P fit and assumption of constant acceleration
# First, the initial velocity is derived by using the velocity and acceleration of the pitch at 50 feet from home plate
# Then the initial time is calculated using the same principles
# After that, the amount of vertical and/or horizontal movement at time = ~0 is determined
# Finally, the angle is derived by observing where the ball is in space at that initial time

def calculate_VRA(vy0, ay, release_extension, vz0, az):
    vy_s = -((vy0**2 - 2 * ay * (60.5 - release_extension - 50)).sqrt())
    t_s = (vy_s - vy0) / ay
    vz_s = vz0 - az * t_s
    VRA = - (vz_s / vy_s).arctan() * (180 / math.pi)
    return VRA

def calculate_HRA(vy0, ay, release_extension, vx0, ax):
    vy_s = -((vy0**2 - 2 * ay * (60.5 - release_extension - 50)).sqrt())
    t_s = (vy_s - vy0) / ay
    vx_s = vx0 - ax * t_s
    HRA = - (vx_s / vy_s).arctan() * (180 / math.pi)
    return HRA

In [ ]:
# Libraries you need to load the data in and make it go fast

import pybaseball as pyb
import pandas as pd
import polars as pl
import math
import numpy as np
from math import sqrt, atan2, degrees, pi, atan
import warnings


In [ ]:
# Uploads every pitch from the 2024 season. It's a lot of data, so this will take a few minutes.
warnings.filterwarnings("ignore", category=FutureWarning)
statcast_data = pyb.statcast(start_dt='2024-03-28', end_dt='2024-09-30')

This is a large query, it may take a moment to complete


/usr/local/lib/python3.10/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
100%|██████████| 187/187 [03:57<00:00,  1.27s/it]


In [ ]:
# Lets you peek at the data to make sure it looks ok
statcast_data.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
202,FF,2024-09-30,97.4,-2.1,4.88,"Díaz, Edwin",518595,621242,field_out,hit_into_play,...,7,7,8,Standard,Strategic,232,-0.142,-0.275,68.81042,7.283
207,SL,2024-09-30,90.7,-2.14,5.06,"Díaz, Edwin",518595,621242,NaN,ball,...,7,7,8,Standard,Strategic,201,0.0,0.041,<NA>,<NA>
210,SL,2024-09-30,91.1,-2.07,5.14,"Díaz, Edwin",518595,621242,NaN,swinging_strike,...,7,7,8,Standard,Strategic,210,0.0,-0.066,71.16004,8.88615
215,SL,2024-09-30,91.3,-2.05,5.07,"Díaz, Edwin",518595,621242,NaN,ball,...,7,7,8,Standard,Strategic,212,0.0,0.013,<NA>,<NA>
222,SL,2024-09-30,89.1,-2.13,5.15,"Díaz, Edwin",518595,621242,NaN,swinging_strike,...,7,7,8,Standard,Standard,216,0.0,-0.045,71.68666,9.03428


In [ ]:
# Drops pitches that are missing important data

statcast_data.dropna(subset=['vy0'], inplace=True)
statcast_data.dropna(subset=['release_extension'], inplace=True)

In [ ]:
# Converts the dataframe from pandas (slow and inefficient) to polars (fast as heck)
statcast_polars = pl.from_pandas(statcast_data)

In [ ]:
# Actually does the calculations, applying the VRA and HRA functions to each pitch
statcast_polars = statcast_polars.with_columns([
    calculate_VRA(pl.col('vy0'), pl.col('ay'), pl.col('release_extension'), pl.col('vz0'), pl.col('az')).alias('VRA'),
    calculate_HRA(pl.col('vy0'), pl.col('ay'), pl.col('release_extension'), pl.col('vx0'), pl.col('ax')).alias('HRA')
])

In [ ]:
# Checking on the data again!

statcast_polars.head()

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,…,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,VRA,HRA
str,datetime[ns],f64,f64,f64,str,i64,i64,str,str,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,i64,str,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,str,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,f64,f64,f64,f64,f64,f64
"""FF""",2024-09-30 00:00:00,97.4,-2.1,4.88,"""Díaz, Edwin""",518595,621242,"""field_out""","""hit_into_play""",null,null,null,null,3,"""Travis d'Arnaud grounds out, s…","""R""","""R""","""R""","""ATL""","""NYM""","""X""",6,"""ground_ball""",2,2,2024,-0.96,0.99,0.67,3.0,null,642201,null,2,9,"""Bot""",…,621242,682626,624413,657193,578428,596019,607043,621438,516782,52.91,0.066,0.058,0.0,1,0,0,2,82,5,"""4-Seam Fastball""",7,8,7,8,8,7,7,8,"""Standard""","""Strategic""",232,-0.142,-0.275,68.81042,7.283,-0.600214,4.141622
"""SL""",2024-09-30 00:00:00,90.7,-2.14,5.06,"""Díaz, Edwin""",518595,621242,null,"""ball""",null,null,null,null,14,"""Travis d'Arnaud grounds out, s…","""R""","""R""","""R""","""ATL""","""NYM""","""B""",null,null,1,2,2024,0.2,0.61,0.75,1.2,null,642201,null,2,9,"""Bot""",…,621242,682626,624413,657193,578428,596019,607043,621438,516782,53.15,null,null,null,null,null,null,null,82,4,"""Slider""",7,8,7,8,8,7,7,8,"""Standard""","""Strategic""",201,0.0,0.041,null,null,-1.99298,3.019111
"""SL""",2024-09-30 00:00:00,91.1,-2.07,5.14,"""Díaz, Edwin""",518595,621242,null,"""swinging_strike""",null,null,null,null,9,"""Travis d'Arnaud grounds out, s…","""R""","""R""","""R""","""ATL""","""NYM""","""S""",null,null,1,1,2024,0.12,0.35,0.66,1.61,null,642201,null,2,9,"""Bot""",…,621242,682626,624413,657193,578428,596019,607043,621438,516782,53.08,null,null,null,null,null,null,null,82,3,"""Slider""",7,8,7,8,8,7,7,8,"""Standard""","""Strategic""",210,0.0,-0.066,71.16004,8.88615,-1.394138,2.927655
"""SL""",2024-09-30 00:00:00,91.3,-2.05,5.07,"""Díaz, Edwin""",518595,621242,null,"""ball""",null,null,null,null,14,"""Travis d'Arnaud grounds out, s…","""R""","""R""","""R""","""ATL""","""NYM""","""B""",null,null,0,1,2024,0.21,0.63,0.61,1.18,null,642201,null,2,9,"""Bot""",…,621242,682626,624413,657193,578428,596019,607043,621438,516782,53.07,null,null,null,null,null,null,null,82,2,"""Slider""",7,8,7,8,8,7,7,8,"""Standard""","""Strategic""",212,0.0,0.013,null,null,-2.096816,2.753169
"""SL""",2024-09-30 00:00:00,89.1,-2.13,5.15,"""Díaz, Edwin""",518595,621242,null,"""swinging_strike""",null,null,null,null,14,"""Travis d'Arnaud grounds out, s…","""R""","""R""","""R""","""ATL""","""NYM""","""S""",null,null,0,0,2024,0.17,0.66,1.36,1.78,null,642201,null,2,9,"""Bot""",…,621242,682626,624413,657193,578428,596019,607043,621438,516782,53.08,null,null,null,null,null,null,null,82,1,"""Slider""",7,8,7,8,8,7,7,8,"""Standard""","""Standard""",216,0.0,-0.045,71.68666,9.03428,-1.401221,3.720652


In [ ]:
# Taking a sample to look at a more manageable selection of the data....

sample_data = statcast_polars.sample(n=100)


In [ ]:
# Converted into an Excel file in case you wanted to scroll through a sample of the pitch data manually

sample_data_pandas = sample_data.to_pandas()

sample_data_pandas.to_excel("sample_data.xlsx", index=False)